<div>
    <h1 style="margin-top: 50px; font-size: 33px; text-align: center"> Homework 5 - Visit the Wikipedia hyperlinks graph! </h1>
    <br>
    <div style="font-weight:200; font-size: 20px; padding-bottom: 15px; width: 100%; text-align: center;">
        <right>Maria Luisa Croci, Livia Lilli, Pavan Kumar Alikana</right>
        <br>
    </div>
    <hr>
</div>

# RQ1

In [ ]:
import pandas as pd
import json
import pickle
from tqdm import tqdm

from collections import defaultdict
from heapq import *
import numpy as np
import collections
import networkx as nx

For our first requests, we can use 2 different approaches:

* We can start from the file building a dictionary that describe our graph; we do it because we will need this dictionary for the request 2;

* Or, better, we can use an easy command <b>nx.info</b>, to get all the informations we need.

So let's see!


## Approach 1

Let's start downloading <a href="https://drive.google.com/file/d/1ghPJ4g6XMCUDFQ2JPqAVveLyytG8gBfL/view">Wikicat hyperlink graph</a> . 

It is a reduced version of the one we can find on SNAP. 

Every row is an <b>edge</b>. The two elements of each row are the <b>nodes</b>, in particular the first is the <b> source</b>, the second represents the <b>destination</b>.

So, our first goal is open and read the file with python, and split each line with the new line charachter.
Then we take all the <i>source nodes</i> for each row, and we put them as keys into our <b>graph</b> dictionary. The values will be all the correspondent destination nodes.

But we have not done! Infact our scope is to analyze the graph, in particular discovering the following informations:

* If it is direct or not;

* The number of nodes;

* The number of edges;

* The average node degree. Is the graph dense?

To do this we want that our dictionary has as keys <u>all the nodes</u>, sources and destinations, and for now we have just the first ones. So we add as new keys all the nodes that are just destinations, putting as values empty lists.


Now we have the dictionary with all the nodes of our graph as keys, and as values there are all the eventual destinations!

In [ ]:
F = open('wiki-topcats-reduced.txt','r') 
all_rows = F.read().split('\n')

graph = {}
for row in all_rows:
    row = row.split("\t")
    if row[0] not in graph:
        try:
            graph[row[0]] = [row[1]]
        except:
            pass
    else:
        graph[row[0]].append(row[1])
    
    

In [ ]:
lista = []
for l in graph.values():
    lista+= l
    

In [ ]:
for node in lista:
    if node not in graph:
        graph[node] = []
    else:
        pass

So, what can we say?

* We are in a case of <b>page ranking</b>. So for definition we have nodes representing sources and destinations, with edges with a particular direction. In other words, our graph has a set of edges which are <i>ordered pairs</i> of nodes, and for the graph theory we have a <b>directed graph</b>.


* The number of nodes is <u>461193</u>. It's just the number of keys of our dictionary.


* The number of edges is <u>2645247</u> and it's computed looking at all the lenghts of the <b>adjacency list</b>.


* In graph theory, the <b>degree</b> (or <i>valency</i>) of a vertex of a graph is the number of edges incident to the vertex. We need the <b>average node degree</b>, so we compute the ratio between number of edges and number of nodes. It results <u>5.735661642739591</u>.

#### Number of nodes

In [ ]:
V = list(graph.keys())
n_nodes = len(V)
n_nodes

#### Number of edges

In [ ]:
n_edges = 0
for l in graph.values():
    n_edges += len(l)
n_edges    

#### Avarage node degree

In [ ]:
avg_degree = n_edges/n_nodes
avg_degree

## Approach 2

Although, since we need the average in and out degree because our graph is directed, we could use an easy command nx.info as follow, in order to obtain the basic informations.

First import the graph from the reduced file of the list of edges indicating with nx.DiGraph that what we want is an oriented graph.

In [ ]:
graph = nx.read_edgelist("wiki-topcats-reduced.txt", delimiter="\t", create_using=nx.DiGraph())
print(nx.info(graph))

**Is the graph dense?**

With the following formula $D=\frac{E}{N(N-1)}$ we obtain a value that could go from 0 up to 1. It measure the probability that any pairs of vertex is connected, so technically if the density is close to 1 the number of edges is close to the maximal number of edges, viceversa if the density is close to 0 we have a graph with only few edges (called sparse graph).



In [ ]:
nx.density(graph)

As we could expect, according to the number of nodes and edges that we already know, the density is very low, so it means that our graph is sparse.

# RQ2

Let's start creating a dictionary called <b>categories</b> where for every category taken from <i>wiki-topcats-categories.txt</i> file, we have the list of all its articles. But attention! We must take into account all the categories that has a number of articles greater than <b>3500</b>. So we filter our dictionary considering the categories with more that 3500 articles. Another, we take just the articles that are the result of the intersection between the set of articles of the category and the articles of our <b>graph</b>; in other words, we don't consider those nodes that are in our graph but not in our categories!


We create also a dictionary called <b>inv_dic</b> that shows for every node (article), a set of all the categories associated. 


In [ ]:
C = open('wiki-topcats-categories.txt','r') 

In [ ]:
categories = {}
for line in C.readlines():
    l = line.split(' ')
    cat = l[0].replace("Category:","").replace(";", "")
    art = l[1:]
    art[-1] = art[-1].replace("\n","")
    if len(art) >= 3500:
        categories[cat]= set(art).intersection(set(V))



In [ ]:
all_set = categories.values()
all_nodes = []
for s in all_set:
    all_nodes += s
inv_dic = {}
for node in all_nodes:
    for cat in categories:
        if node in categories[cat] and node not in inv_dic:
            inv_dic[node] = [cat]
        elif node in categories[cat] and node in inv_dic and cat not in inv_dic[node]:
            inv_dic[node].append(cat)
        else:
            pass


## Block Ranking 

Our scope now is, to take in input a category $C_0 = \{article_1, article_2, \dots \}$. Then we want to rank all of the nodes according to the following criterion:

Obtain a <b>block-ranking</b>, where the blocks are represented by the categories.
The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by: $$distance(C_0, C_1) = median(ShortestPath(C_0, C_i))$$

How we do that? At first we create the functions we need.

Our input category is 'Year_of_birth_unknown' for convention because the one with the smaller number of nodes.

* The first function we write is the <b> ShortestPath</b> which takes in input a node (of the input category) and our graph. It computes the distances, using the visit in amplitude of the graph. For this we apply the <b><i>BFS</i></b> algorithm, that consists in searching graph data structures. It starts at the <i>tree root</i> (or some arbitrary node of a graph called <i>search key</i>), and at first it explores all of the neighbor nodes at the present depth prior, moving on to the nodes at the next depth level. The gif below shows this concept.

So the SorthestPath function creates a dictionary where the keys are the nodes (including the input node) and their value are the distances from the node of the input category. 

The distance from the node of the input category to itself is written as zero. The others are started as -1, and then eventually incremented.


* Now it's the turn of <b>createDistancesDict</b> function, which take 4 elements as input: the input category, the graph, the <i>categories</i> dictionary and finally the <i>inv_dic</i>. In easy words, it applies the ShortestPath function to every node of the input cateogory creating a dictionary where each key is one of these nodes, and the value is a dictionary where for every other node of the graph there is the distance from the starting node of C0.


* Now we create the <b>dictDistanceCi</b> dictionary, where we wanna show for each category a list of all the distances of the correspondent nodes from the nodes of the input category. Of course we don't need the distances among the nodes of the input cateogory, so we don't consider them.


* A the end of our process, we compute for each category (taken from the precedent dictionary) the <b>median</b> of the correspondent distances. Then we add in an Ordered Dictionary called <b>rank</b> each category with its value of median. So we obtain our <b>BLOCK RANKING</b>.




<img src="https://upload.wikimedia.org/wikipedia/commons/5/5d/Breadth-First-Search-Algorithm.gif">

In [ ]:
input_category = input()


In [ ]:
def ShortestPath(c0, graph):
    queue = []
    queue.append(c0)
    
    distanceDict = dict()
    for node in graph:
        distanceDict[node] = -1
    distanceDict[c0] = 0

    while queue:
        vertex = queue.pop(0)
        for i in graph[vertex]:
            if distanceDict[i] == -1:
                queue.append(i)
                distanceDict[i] = distanceDict[vertex] + 1
    return distanceDict
    

In [ ]:
def calculateMedian(lista):
    lung = len(lista)
    #ordinata = sorted(lista)
    ordinata = lista
    if(lung % 2 != 0):
        return ordinata[lung/2]
    else:
        return (ordinata[lung/2]) + (ordinata[lung/2 + 1]) /2 

In [ ]:
from collections import OrderedDict
import pickle

def createDistancesDict(c0, graph, dizionarioCatNodi, listNode):
    
    #listNode è un dizionario <articolo, [categorie]>
    
    #Prendo come categoria 0 la lista di nodi della categoria 0
    Category0 = dizionarioCatNodi[c0]
    
    #Dizionario dove per ogni chiave(articolo in C0) c'è un dizionatio (nodo, distanza) con la distanza verso tutti gli altri nodi 
    dictDistances = dict()
    
    for node in tqdm(Category0):
        try:
            dictDistances[node] = ShortestPath(node, graph)
        except Exception as e: print(e)
    
    with open("distance_dict.p", 'wb') as handle:
        pickle.dump(dictDistances, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
createDistancesDict(input_category, graph, categories, inv_dic)

In [ ]:
with open("distance_dict.p", 'rb') as handle:
    dist_dict = pickle.load(handle)

In [ ]:
dictDistanceCi = dict()
#inizializzo le distanze da C0 per ogni categoria ad una lista vuota
for cat in categories:
    dictDistanceCi[cat] = []

In [ ]:
#for every cat the distances of its nodes from nodes of C0
for node in dist_dict:
    for node2 in dist_dict[node]:
        for cat in inv_dic[node2]:
            if cat != inv_dic[node]:
                dictDistanceCi[cat].append(dist_dict[node][node2])

with open("dictDistanceCi.p", 'ab') as handle:
    pickle.dump(dictDistanceCi, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("dictDistanceCi.p", 'rb') as handle:
    dictDistanceCi = pickle.load(handle)

In [ ]:
rank = OrderedDict()
for cat in tqdm(dictDistanceCi):
    distance = np.median(dictDistanceCi[cat])
    rank[cat] = distance

rank['Year_of_birth_unknown'] = 0

In [ ]:
block_rank = {}
for tupla in rank:
    block_rank[tupla[0]] = tupla[1]

In [ ]:
for el in block_rank:
    if block_rank[el] == -1.0:
        block_rank[el] = 10000.0
block_rank = sorted(block_rank.items(), key=lambda x: x[1])

In [ ]:
block_rank

Obtained the Ordered Dictionary <b>rank</b> we notice that there are some categories with median equal to -1. This means that these categories are not reachable from the input category and so the values of distance among their nodes and the input category ones didn't change its initial values -1 associated during the inizialization in the BFS. For this reason we give to them a big value, for example 10000, so that in the block rank they will stay at the end.

## Sorting nodes of category

Once we obtain the <i>block ranking vector</i>, we want to sort the nodes in each category. The way we should sort them is the following...

 We have to compute the subgraph induced by $C_0$. Then, for each node, we compute the sum of the weigths of the <b>in-edges</b>. The nodes will be ordered by this score.
 The following image explains how to do it for each step.
 
 In other words, we have to consider a category, and for that category we must compute for each node the number of in-edges, but considering just those that have the source of the same category! For example, in the first image, the B node of the category "0" has got 2 in-edges, but only one is from a node of the same category.

<img src="https://raw.githubusercontent.com/CriMenghini/ADM-2018/master/Homework_5/imgs/algorithm.PNG">

For this scope we have created a function called <b>in_edges</b> that implements our idea of sorting, given as input a category. 

So we apply this function for each category saving the correspondent dictionary on a file <i>pickle</i>, naming it as <i>"cat_i.p"</i> where i is the index of the i-category. To control the correspondence index-category, we create a dictionary where for each category we have its index; we call it <b>indexing</b>. 

What does our <i>in_edge()</i> function do exactly? 

Well, we can see that for a node <i>n1</i> of the choosen category, it starts a contator and for every node <i>n2</i> of our graph checks two important things:

* if there is an edge from <i>n2</i> to <i>n1</i>;

* if <i>n2</i>, the source node, is in the same category of <i>n1</i>;

If these 2 points are respected, then it increments the contator of <i>n1</i>. 

At the end, it saves in a dictionary each node n1 and its contator, or in other words, the number of its in-edges.
But it's not finished! We want to sort the nodes in the dictionary in base of their values, so we just do it. Now the output is ready!


We have reported as examples some of our  dictionaries saved on pickle. In particular you can see that for "the category 7"  (that in our block ranking correponds to the <b>Category0</b>).

In [ ]:
all_cat = list(categories.keys())

In [ ]:
def in_edges(cat, graph):
    n_cat = categories[cat]
    d = {}
    for n1 in tqdm(n_cat):
        count = 0
        for n2 in graph:
            if n1 in graph[n2] and n2 in n_cat:
                count += 1
        d[n1] = count
    d = sorted(d.items(), key=lambda x: x[1])
    return d
    

In [ ]:
for i in range(len(all_cat)):
    dd = in_edges(all_cat[i], graph)
    
    #pickle.dump(dd, open( "cat"+str(i)+".p", "wb" ) )
    with open("cat"+str(i)+".p", 'wb') as handle:
        pickle.dump(dd, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
indexing = {}
for i in range(len(all_cat)):
    indexing[all_cat[i]] = i
    

In [ ]:
indexing

Here there is the indexing dictionary, that occurs to us to find the in_edge dictionary of a particular category, starting from its index.

Here, as promised, we have the dictionary for the category 0 of our block ranking, or in other words, the category7 of our indexing.

For convention we print just a portion of it, in particular a part where we can see the moment where the score changes.

In [ ]:
with open("cat"+str(7)+".p", 'rb') as handle:
    dd7 = pickle.load(handle)

In [ ]:
print(dd7[1600:1700])


<img src="http://scalar.usc.edu/works/querying-social-media-with-nodexl/media/Network_theoryarticlenetworkonWikipedia1point5deg.jpg" height="200" width="400">